# Creational Patterns

Creational patterns deal with the creation/construction of objects:

- The creation can be explicit (e.g., via constructors) of implicit (e.g., dependency injection, reflection, etc.).
- The creation can be wholesale (single statement creates the object) or piecewise, i.e., it is done step-by-step in a more complicated process.

Table of contents:

- [Builder](#Builder)
- [Factories](#Factories)
- [Prototype](#Prototype)
- [Singleton](#Singleton)

## Builder

Sometimes objects are easy to create and they are built witha single initializer call. However, in some other cases creating an object is more complicated

- it requires several steps
- or it dozens of parameters are involved
- or a piecewise construction is required.

The **Builder** provides an API for constructing an object step-by-step: when piecewise object construction is complicated, it provides an API for doing it succinctly.

### Example: HTML Builder

In [1]:
# If we want to build a simple HTML paragraph using a list
hello = 'hello'
parts = ['<p>', hello, '</p>']
print(''.join(parts))

<p>hello</p>


In [2]:
# Now I want an HTML list with 2 words in it
words = ['hello', 'world']
parts = ['<ul>']
for w in words:
    parts.append(f'  <li>{w}</li>')
parts.append('</ul>')
print('\n'.join(parts))

<ul>
  <li>hello</li>
  <li>world</li>
</ul>


We see that the complexity explodes as we start having rich HTML pages with different components. Solution, we define classes that carry out the building task together: `HtmlElement` and `HtmlBuilder`.

In [1]:
class HtmlElement:
    indent_size = 2

    def __init__(self, name="", text=""):
        self.name = name # div, p, etc.
        self.text = text
        self.elements = [] # we will populate this list

    def __str(self, indent):
        """This function recursively calls itself to produce
        the HTML page text:
        
        <{name}>
        {text}
            <{name}>
            {text}
                <{name}>
                {text}
                    ...
                </{name}>                
            </{name}>
        </{name}>
        """
        lines = []
        i = ' ' * (indent * self.indent_size)
        lines.append(f'{i}<{self.name}>')

        if self.text:
            i1 = ' ' * ((indent + 1) * self.indent_size)
            lines.append(f'{i1}{self.text}')

        for e in self.elements:
            # ATTENTION: recursive call with updated indentation
            lines.append(e.__str(indent + 1))

        lines.append(f'{i}</{self.name}>')
        return '\n'.join(lines)

    def __str__(self):
        # The goal is to print the HTML element
        # but for that we need the correct indentation.
        # To that end, we call an internal __str()
        # which takes the correct indentation
        return self.__str(0)

    # This is added at the end, as an add on.
    # In some way this is breaking the Open-Close principle
    # and it seems a circular dependency injection.
    # However, it allows to give us an alternative to just use
    # HtmlElement only, i.e., the user interacts only with
    # HtmlElement and not HtmlBuilder.
    # Keep in mind that HtmlBuilder is used behind the scenes all the time
    # but the user would have to invoke it only in the beginning.
    # Alternative solution: use a factory, shown later. 
    @staticmethod
    def create(name):
        return HtmlBuilder(name)

In [7]:
class HtmlBuilder:
    #__root = HtmlElement()

    def __init__(self, root_name):
        self.__root = HtmlElement()
        self.root_name = root_name
        self.__root.name = root_name

    # Building API: ordinary, not fluent
    def add_child(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )

    # Building API: fluent
    # A fluent API allows to chain calls one after another:
    # builder.add_child_fluent(...).add_child_fluent(...)
    def add_child_fluent(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )
        # Fluent APIs are created by returning self
        return self

    def clear(self):
        self.__root = HtmlElement(name=self.root_name)

    def __str__(self):
        return str(self.__root)

In [11]:
# Ordinary non-fluent builder
#builder = HtmlBuilder('ul')
builder = HtmlElement.create('ul')
builder.add_child('li', 'hello')
builder.add_child('li', 'world')
print('Ordinary builder:')
print(builder)

Ordinary builder:
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


In [12]:
# Fluent builder
builder.clear()
(builder
    .add_child_fluent('li', 'hello')
    .add_child_fluent('li', 'world')
)
print('Fluent builder:')
print(builder)

Fluent builder:
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


In [16]:
# This alternative would allow us to remove the static method create()
# from HtmlElement
class HtmlElementFactory:
    @staticmethod
    def create(name):
        return HtmlBuilder(name)

# Usage
builder = HtmlElementFactory.create('ul')
builder.add_child_fluent('li', 'hello').add_child_fluent('li', 'world')
print(builder)

<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


### Example: Car Builder

In [14]:
class Car:
    def __init__(self):
        self.features = []

    def add_feature(self, feature):
        self.features.append(feature)

    def __str__(self):
        return ', '.join(self.features)

class CarBuilder:
    def __init__(self):
        self.car = Car()

    def add_engine(self, engine):
        self.car.add_feature(f"Engine: {engine}")
        return self

    def add_wheels(self, wheels):
        self.car.add_feature(f"Wheels: {wheels}")
        return self

    def add_gps(self):
        self.car.add_feature("GPS: Installed")
        return self

    def build(self):
        return self.car

class Director:
    def construct_sports_car(self, builder):
        return builder.add_engine('V8').add_wheels('Sports').add_gps().build()

    def construct_family_car(self, builder):
        return builder.add_engine('V4').add_wheels('Regular').add_gps().build()

# Usage
director = Director()
sports_car_builder = CarBuilder()
family_car_builder = CarBuilder()

sports_car = director.construct_sports_car(sports_car_builder)
family_car = director.construct_family_car(family_car_builder)

print(sports_car)  # Engine: V8, Wheels: Sports, GPS: Installed
print(family_car)  # Engine: V4, Wheels: Regular, GPS: Installed


Engine: V8, Wheels: Sports, GPS: Installed
Engine: V4, Wheels: Regular, GPS: Installed


### Example: Person Builder